In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
  from langchain_core.prompts import PromptTemplate
except ImportError:
  !pip install langchain_core
  from langchain_core.prompts import PromptTemplate

try:
  from langchain_openai import ChatOpenAI
except ImportError:
  !pip install langchain_openai
  from langchain_openai import ChatOpenAI

try:
  import chromadb
except ImportError:
  !pip install chromadb
  import chromadb

try:
  from langchain_chroma import Chroma
except ImportError:
  !pip install langchain_chroma
  from langchain_chroma import Chroma

try:
  from langchain_openai import OpenAIEmbeddings
except ImportError:
  !pip install langchain_openai
  from langchain_openai import OpenAIEmbeddings

try:
  from langchain_core.documents import Document
except ImportError:
  !pip install langchain_core
  from langchain_core.documents import Document

In [ ]:
try:
  from langchain_core.prompts import PromptTemplate
except ImportError:
  !pip install langchain_core
  from langchain_core.prompts import PromptTemplate

try:
  from langchain_openai import ChatOpenAI
except ImportError:
  !pip install langchain_openai
  from langchain_openai import ChatOpenAI

try:
  import chromadb
except ImportError:
  !pip install chromadb
  import chromadb

try:
  from langchain_chroma import Chroma
except ImportError:
  !pip install langchain_chroma
  from langchain_chroma import Chroma

try:
  from langchain_openai import OpenAIEmbeddings
except ImportError:
  !pip install langchain_openai
  from langchain_openai import OpenAIEmbeddings

try:
  from langchain_core.documents import Document
except ImportError:
  !pip install langchain_core
  from langchain_core.documents import Document

In [ ]:
try:
    from langchain_classic.retrievers.multi_query import MultiQueryRetriever
except ImportError:
    !pip install langchain_classic
    from langchain_classic.retrievers.multi_query import MultiQueryRetriever


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

llm = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.01,
    max_tokens = 500,
    openai_api_key = userdata.get('OPENAI_API_KEY')
)

prompt_multi_query = """
질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.
각 쿼리는 반드시 줄바꿈으로 구분해.

질문: {question}
"""


In [ ]:
db_path = "/content/drive/MyDrive/rag/chromadb_backup"
client = chromadb.PersistentClient(path=db_path)
collection_name = "tax_law"

try:
  collection = client.get_collection(name=collection_name)
except Exception:
  raise RuntimeError(f"Collection {collection_name} not found")


In [ ]:
embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model, openai_api_key = userdata.get("OPENAI_API_KEY"))

vectorstore = Chroma(client = client, collection_name=collection_name, embedding_function=embeddings)


In [ ]:
try:
  from langchain_core.output_parsers import StrOutputParser
except ImportError:
  !pip install langchain_core
  from langchain_core.output_parsers import StrOutputParser

In [ ]:
class LLMRerank:

  def __init__(self):
    self.llm = ChatOpenAI(model="gpt-4.1-nano", openai_api_key=userdata.get("OPENAI_API_KEY"), max_tokens = 200)

    self.prompt = """
    너는 세법 전문가야.
    질문에 대해 가장 정확한 '법률적 정의'와 '계산 근거', '법률적 조건'을 포함하고 있는 상위 {final_k}개의 문서를 나열해.
    질문과 단어가 겹치는 문서보다, 질문의 '의도'를 해결해주는 문서를 우선해.
    반드시 상위 {final_k}개의 문서 번호만 쉼표로 구분해서 답변해.

    질문: {query}
    문서: {docs}

    ex. 문서 번호 1,2,3,4,5 (final_k = 3)인 경우 3,1,4로 답변 (쉼표로 번호 구분해서 답변)
    답변:
    """

  def rerank(self, query, docs, final_k):
    docs_for_reranking = [f"[{i}] {doc.page_content}" for i, doc in enumerate(docs)]
    docs_for_reranking = "\n\n".join(docs_for_reranking)

    reranking_prompt = PromptTemplate.from_template(self.prompt)

    chain = reranking_prompt | self.llm | StrOutputParser()

    reranked_docs = chain.invoke({"final_k": final_k, "query": query, "docs": docs_for_reranking})

    reranked_docs = reranked_docs.split(",")

    reranked_docs = [int(x.strip()) for x in reranked_docs]

    final_docs = []
    seen = set()
    for x in reranked_docs:
      if x not in seen:
        seen.add(x)
        final_docs.append(docs[x])

      if len(final_docs) >= final_k:
        break

    return final_docs



In [ ]:
import json
import re
from langchain_core.output_parsers import JsonOutputParser

class LLMRerank2:
  # LLM이 각 문서별로 질문과 관련된 정도에 대해 점수를 매김

  def __init__(self):
    self.llm = ChatOpenAI(model="gpt-4.1-nano", openai_api_key=userdata.get("OPENAI_API_KEY"), max_tokens = 1000)

    self.prompt = """
    너는 세법 전문 지식을 가진 reranking 모델이야.
    아래 질문에 대해 문서들 중 답변 근거로 가장 적절하고 관련성 있는 상위 {top_k}개의 문서를 골라줘.
    id는 반드시 제공된 [문서 번호] 숫자만 사용하고, 상위 {top_k}개의 문서에 대해 점수(0~100)을 매겨서 JSON 형식으로 답변해

    - 질문과 직접 관련되고 법적 근거로 적합할수록 높은 점수
    - 미세하게라도 점수에 차등을 둘 것
    - 문서 간 비교를 하여 상대적인 점수를 계산할 것

    [질문]
    {query}

    [문서들]
    {docs}

    [출력 형식-JSON]
    {{
      "scores": [
        {{"id": 0, "score": 95}},
        {{"id": 1, "score": 80}},
        ...
      ]
    }}
    """

  def rerank(self, query, docs, final_k):

    docs_matching = {} # docs 순서의 인덱싱
    for i, doc in enumerate(docs):
      docs_matching[str(i)] = doc

    docs_formatted = "\n".join([f"[문서 ID: {i}] {doc.page_content[:500]}..." for i, doc in docs_matching.items()])

    prompt = PromptTemplate.from_template(self.prompt)

    chain = prompt | self.llm | JsonOutputParser()

    try:
      result = chain.invoke({"query": query, "docs": docs_formatted, "top_k": final_k})
      scores = result.get("scores", [])

      scored_docs = []
      for x in scores:
        id = str(x.get("id"))
        score = int(x.get("score", 0))

        if id in docs_matching:
          scored_docs.append((docs_matching[id], score))

      scored_docs.sort(key=lambda x: x[1], reverse=True)

      reranked_docs = [doc for doc, _ in scored_docs[:final_k]]

      return reranked_docs

    except Exception as e:
      print(f"reranking 오류: {e}")
      return docs[:final_k]







In [ ]:
import json
import re
from langchain_core.output_parsers import JsonOutputParser

class LLMRerank2:
  # LLM이 각 문서별로 질문과 관련된 정도에 대해 점수를 매김

  def __init__(self):
    self.llm = ChatOpenAI(model="gpt-4.1-nano", openai_api_key=userdata.get("OPENAI_API_KEY"), max_tokens = 1000)

    self.prompt = """
    너는 세법 전문 지식을 가진 reranking 모델이야.
    아래 질문에 대해 문서들 중 답변 근거로 가장 적절하고 관련성 있는 상위 {top_k}개의 문서를 골라줘.
    id는 반드시 제공된 [문서 번호] 숫자만 사용하고, 상위 {top_k}개의 문서에 대해 점수(0~100)을 매겨서 JSON 형식으로 답변해

    - 질문과 직접 관련되고 법적 근거로 적합할수록 높은 점수
    - 미세하게라도 점수에 차등을 둘 것
    - 문서 간 비교를 하여 상대적인 점수를 계산할 것

    [질문]
    {query}

    [문서들]
    {docs}

    [출력 형식-JSON]
    {{
      "scores": [
        {{"id": 0, "score": 95}},
        {{"id": 1, "score": 80}},
        ...
      ]
    }}
    """

  def rerank(self, query, docs, final_k):

    docs_matching = {} # docs 순서의 인덱싱
    for i, doc in enumerate(docs):
      docs_matching[str(i)] = doc

    docs_formatted = "\n".join([f"[문서 ID: {i}] {doc.page_content[:500]}..." for i, doc in docs_matching.items()])

    prompt = PromptTemplate.from_template(self.prompt)

    chain = prompt | self.llm | JsonOutputParser()

    try:
      result = chain.invoke({"query": query, "docs": docs_formatted, "top_k": final_k})
      scores = result.get("scores", [])

      scored_docs = []
      for x in scores:
        id = str(x.get("id"))
        score = int(x.get("score", 0))

        if id in docs_matching:
          scored_docs.append((docs_matching[id], score))

      scored_docs.sort(key=lambda x: x[1], reverse=True)

      reranked_docs = [doc for doc, _ in scored_docs[:final_k]]

      return reranked_docs

    except Exception as e:
      print(f"reranking 오류: {e}")
      return docs[:final_k]







In [ ]:
from langchain_core.globals import set_debug

set_debug(False)

In [ ]:
import re
from langchain_core.output_parsers import JsonOutputParser

class MultiQueryPipeline:
  def __init__(self, prompt_multi_query, llm, vectorstore,initial_k=25, final_k=10):
    self.prompt = PromptTemplate(
        input_variables=["question"],
        template=prompt_multi_query
    )

    self.llm = llm
    self.initial_k = initial_k
    self.final_k = final_k

    basic_retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": self.initial_k})

    self.multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever = basic_retriever,
        llm = self.llm,
        prompt = self.prompt,
        include_original = True
    )

  def llm_reranking(self, query):
    docs = self.multiquery_retriever.invoke(query)

    reranker = LLMRerank2()
    reranked_docs = reranker.rerank(query, docs, self.final_k)

    # id 매핑
    idx_to_id = {str(i): getattr(doc, "id", "None") for i, doc in enumerate(docs)}

    #context = "\n\n".join(f"[문서 ID: {getattr(doc, "id", None)}\n{doc.page_content}" for doc in reranked_docs)
    context = "\n\n".join(f"[[{i}]]\n{doc.page_content}" for i, doc in enumerate(reranked_docs)) # reranked docs 순서의 인덱싱 (점수 순)
    print(context)

    return context, reranked_docs



  def generate_answer(self, query):
    context, reranked_docs = self.llm_reranking(query)

    prompt_template = """
    너는 회계, 세법 전문가야. 아래의 context를 근거로 다음 규칙에 따라 질문에 답변해.

    [규칙]
    1. 결론: 질문에 대한 답은 한 문장으로 먼저 제시 (ex. "납부할 세액은 1,000,000원입니다." "해당 항목은 익금산입 대상입니다.")
    2. 출처는 context에등장하는 [문서 ID: ...]의 값만 그대로 복사해서 사용. (요약, 생략 불가)

    [문맥]
    {context}

    [질문]
    {question}

    <출력 형식>
    [답변]
    1. 결론: ...
    2. 결론 도출 과정: ...

    [출처]
    ["문서 ID1", "문서 ID2", ...]
    - 답변 생성에 사용한 문서의 ID만 작성
    - 1 ~ {final_k}개의 문서 ID 작성
    """

    prompt = PromptTemplate(input_variables = ["context", "question", "final_k"], template = prompt_template)

    chain = prompt | self.llm | StrOutputParser()

    answer = chain.invoke({"context": context, "question": query, "final_k": self.final_k})

    return answer, reranked_docs

  def generate_answer2(self, query):
    context, reranked_docs = self.llm_reranking(query)

    prompt_template = """
    너는 세법 전문가야. 아래의 문백을 바탕으로 질문에 답변해.
    반드시 아래의 JSON 형식을 지켜서 출력해

    [문맥]
    {context}

    [질문]
    {question}

    [출력 형식-JSON]
    {{
      "answer": "질문에 대한 답변 (결론 선제시, 논리적 근거 설명)",
      "used_ids": ["0", "2"]
    }}
    """

    prompt = PromptTemplate(input_variables = ["context", "question"], template = prompt_template)
    chain = prompt | self.llm | JsonOutputParser()
    answer = chain.invoke({"context": context, "question": query})

    return answer, reranked_docs

  def strip_dot(self, text):
    if not text:
      return ""
    return text.rstrip(".")

  def clean_content(self, text):
    if not text:
      return text

    text = text.strip()

    text = re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩]+\s*", "", text)
    text = re.sub(r"^\d+\.\s*", "", text)
    text = re.sub(r"^[가-힣]\.\s*", "", text)

    return text.strip()

  def make_reference(self, doc, max_law_chars):
    출처 = ""
    meta = doc.metadata
    law_name = meta.get("law_name")
    출처 += law_name
    조문번호 = meta.get("조문번호", "")
    출처 += f" {조문번호}조" if 조문번호 else ""
    항번호 = meta.get("항번호", "")
    출처 += f" {self.strip_dot(항번호)}항" if 항번호 else ""
    호번호 = meta.get("호번호", "")
    출처 += f" {self.strip_dot(호번호)}호" if 호번호 else ""
    목번호 = meta.get("목번호", "")
    출처 += f" {self.strip_dot(목번호)}목" if 목번호 else ""

    content = self.clean_content(doc.page_content)
    if len(content) > max_law_chars:
      content = content[:max_law_chars] + "..."

    return 출처, content


  def final_output(self,query, max_law_chars=400):
    answer, reranked_docs = self.generate_answer2(query)

    answer_text = answer.get("answer", "")
    used_ids = answer.get("used_ids", [])

    reference_list = []
    for i in used_ids:
      try:
        i = int(i)
        doc = reranked_docs[i]
        출처, content = self.make_reference(doc, max_law_chars)
        reference_list.append(f"{출처}: {content}")
      except (IndexError, ValueError):
        continue

    final_output = answer_text + "\n\n[근거 법령 및 본문]\n" + "\n\n".join(reference_list)

    return final_output

In [ ]:
query = "예식장 등에서 다른 사업자가 제공하는 용역(예: 개인 미용실)에 대하여 소비자와 일괄계약을 체결하고 그 대금을 지급받은 경우는 어떻게 하나요?"

print(MultiQueryPipeline(prompt_multi_query, llm, vectorstore).final_output(query))

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "예식장 등에서 다른 사업자가 제공하는 용역(예: 개인 미용실)에 대하여 소비자와 일괄계약을 체결하고 그 대금을 지급받은 경우는 어떻게 하나요?"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "예식장 등에서 다른 사업자가 제공하는 용역(예: 개인 미용실)에 대하여 소비자와 일괄계약을 체결하고 그 대금을 지급받은 경우는 어떻게 하나요?"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.\n각 쿼리는 반드시 줄바꿈으로 구분해.\n\n질문: 예식장 등에서 다른 사업자가 제공하는 용역(예: 개인 미용실)에 대하여 소비자와 일괄계약을 체결하고 그 대금을 지급받은 경우는 어떻게 하나요?"
  ]
}
[llm/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] [1.31s] Exiting LLM run with output:
{
  "gene

In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
print(MultiQueryPipeline(prompt_multi_query, llm, vectorstore).final_output(query))

[[0]]
② 근로소득이 있는 거주자가 기본공제대상자(나이 및 소득의 제한을 받지 아니한다)를 위하여 해당 과세기간에 대통령령으로 정하는 의료비를 지급한 경우 다음 각 호의 금액의 100분의 15(제3호의 경우에는 100분의 20, 제4호의 경우에는 100분의 30)에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. <개정 2014.12.23, 2016.12.20, 2017.12.19, 2021.12.8, 2023.12.31>

[[1]]
2. 거주자의 배우자로서 해당 과세기간의 소득금액이 없거나 해당 과세기간의 소득금액 합계액이 100만원 이하인 사람(총급여액 500만원 이하의 근로소득만 있는 배우자를 포함한다)

[[2]]
① 양도소득이 있는 거주자에 대해서는 다음 각 호의 소득별로 해당 과세기간의 양도소득금액에서 각각 연 250만원을 공제한다. <개정 2014.12.23, 2020.12.29, 2024.12.31>

[[3]]
1의2. 법 제21조제1항제7호ㆍ제8호의2ㆍ제9호ㆍ제15호 및 제19호의 기타소득에 대해서는 거주자가 받은 금액의 100분의 70(2019년 1월 1일이 속하는 과세기간에 발생한 소득분부터는 100분의 60)에 상당하는 금액을 필요경비로 한다. 다만, 실제 소요된 필요경비가 거주자가 받은 금액의 100분의 70(2019년 1월 1일이 속하는 과세기간에 발생한 소득분부터는 100분의 60)에 상당하는 금액을 초과하면 그 초과하는 금액도 필요경비에 산입한다.

[[4]]
3. 법 제4조에 따른 소득 중 재정경제부령으로 정하는 소득이 「국민기초생활 보장법」 제2조제11호에 따른 기준 중위소득을 12개월로 환산한 금액의 100분의 40 수준 이상으로서 소유하고 있는 주택 또는 토지를 관리ㆍ유지하면서 독립된 생계를 유지할 수 있는 경우. 다만, 미성년자의 경우를 제외하되, 미성년자의 결혼, 가족의 사망 그 밖에 재정경제부령이 정하는 사유로 1세대의 구성이 불가피한 경우에는 그러하지 아니하다.

[[5]]
3. 「조세특례제한법」

In [ ]:
class Reference2:

  def strip_dot(self, text):
    if not text:
      return ""
    return text.rstrip(".")

  def clean_content(self, text):
    if not text:
      return text

    text = text.strip()

    text = re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩]+\s*", "", text)
    text = re.sub(r"^\d+\.\s*", "", text)
    text = re.sub(r"^[가-힣]\.\s*", "", text)

    return text.strip()

  def make_reference(self, doc, max_law_chars):
    출처 = ""
    meta = doc.metadata
    law_name = meta.get("law_name")
    출처 += law_name
    조문번호 = meta.get("조문번호", "")
    출처 += f" {조문번호}조" if 조문번호 else ""
    항번호 = meta.get("항번호", "")
    출처 += f" {self.strip_dot(항번호)}항" if 항번호 else ""
    호번호 = meta.get("호번호", "")
    출처 += f" {self.strip_dot(호번호)}호" if 호번호 else ""
    목번호 = meta.get("목번호", "")
    출처 += f" {self.strip_dot(목번호)}목" if 목번호 else ""

    content = self.clean_content(doc.page_content)
    if len(content) > max_law_chars:
      content = content[:max_law_chars] + "..."

    return 출처, content

  def final_output(self,query, pipeline, max_law_chars=400):
    answer, reranked_docs = pipeline.generate_answer2(query)

    answer_text = answer.get("answer", "")
    used_ids = answer.get("used_ids", [])

    docs_dict = {getattr(doc, "id", str(i)): doc for i, doc in enumerate(reranked_docs)}

    reference_list = []
    for id in used_ids:
      doc = docs_dict.get(id)
      if not doc:
        matched_key = next((k for k in docs_dict.keys() if id in k or k in id), None)
        doc = docs_dict.get(matched_key)

      if doc:
        출처, content = self.make_reference(doc, max_law_chars)
        reference_list.append(f"{출처}: {content}")

    final_output = answer_text + "\n\n[근거 법령 및 본문]\n" + "\n\n".join(reference_list)

    return final_output


In [ ]:
ㅈquery = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"

pipeline = MultiQueryPipeline(prompt_multi_query, llm, 25, vectorstore)
print(Reference2().final_output(query, pipeline))

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.\n각 쿼리는 반드시 줄바꿈으로 구분해.\n\n질문: 소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
  ]
}
[llm/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] [3.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. 소득세법상 부양가족 공제의 연간 소득금액 기준이 10

In [ ]:
query = "미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"

pipeline = MultiQueryPipeline(prompt_multi_query, llm, 25, vectorstore)
print(Reference2().final_output(query, pipeline))

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.\n각 쿼리는 반드시 줄바꿈으로 구분해.\n\n질문: 미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"
  ]
}
[llm/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] [2.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. 미혼자가 본인 명의로 임대한 주택에 

In [ ]:
import re

class Reference:

  def strip_dot(self, text):
    if not text:
      return ""
    return text.rstrip(".")

  def clean_content(self, text):
    if not text:
      return text

    text = text.strip()

    text = re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩]+\s*", "", text)
    text = re.sub(r"^\d+\.\s*", "", text)
    text = re.sub(r"^[가-힣]\.\s*", "", text)

    return text.strip()

  def make_reference(self, doc, max_law_chars):
    출처 = ""
    meta = doc.metadata
    law_name = meta.get("law_name")
    출처 += law_name
    조문번호 = meta.get("조문번호", "")
    출처 += f" {조문번호}조" if 조문번호 else ""
    항번호 = meta.get("항번호", "")
    출처 += f" {self.strip_dot(항번호)}항" if 항번호 else ""
    호번호 = meta.get("호번호", "")
    출처 += f" {self.strip_dot(호번호)}호" if 호번호 else ""
    목번호 = meta.get("목번호", "")
    출처 += f" {self.strip_dot(목번호)}목" if 목번호 else ""

    content = self.clean_content(doc.page_content)
    if len(content) > max_law_chars:
      content = content[:max_law_chars] + "..."

    return 출처, content

  def match_id(self, cited_id, docs_id):
    if cited_id in docs_id:
      return cited_id

    # cited_id = str(cited_id)

    for k in docs_id:
      # k = str(k)
      if k.startswith(cited_id) or cited_id.startswith(k):
        return k

  def final_output(self, query, pipeline, max_law_chars=400, k_preliminary=2):
    answer, reranked_docs = pipeline.generate_answer(query)

    docs_list = {}
    for i, doc in enumerate(reranked_docs, 1):
      doc_id = getattr(doc, "id", None)
      if doc_id is None:
        doc_id = f"NO_ID_doc{i}"

      docs_list[doc_id] = doc

    ids = re.findall(r"ID:([^\s,]+)", answer)
    print("ids: ", ids)

    answer = re.sub(r"\n?\[출처\][\s\S]*$", "", answer.strip()).strip()

    if not ids:
      ids = list(docs_list.keys())[:k_preliminary]

    used_docs = []
    seen = set()
    docs_id = list(docs_list.keys())

    for id in ids:
      matched = self.match_id(id, docs_id)
      if matched is not None:
        used_docs.append(docs_list[matched])
        seen.add(matched)

    print("docs_id", docs_id)
    print("used_docs", used_docs)

    reference = []
    for doc in used_docs:
      출처, content = self.make_reference(doc, max_law_chars)
      reference.append(f"{출처}: {content}")

    final_output = answer + "\n\n[근거 법령 및 본문]\n" + "\n\n".join(reference)

    return final_output





In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"

print(MultiQueryPipeline(prompt_multi_query, llm, 25, 10, vectorstore).generate_answer(query))



①피상속인의 소득금액에 대한 소득세로서 상속인에게 과세할 것과 상속인의 소득금액에 대한 소득세는 구분하여 계산하여야 한다. <개정 2013.1.1>

나. 「소득세법」 제17조제1항 각 호에 따른 배당소득의 금액. 다만, 「상속세 및 증여세법」 제16조 또는 제48조에 따라 상속세 과세가액 또는 증여세 과세가액에 산입되거나 증여세가 부과되는 주식등으로부터 발생한 배당소득의 금액은 제외한다.

⑤ 자산양도소득에 대한 과세표준의 계산에 관하여는 「소득세법」 제101조 및 제102조를 준용하고, 자산양도소득에 대한 세액계산에 관하여는 같은 법 제92조를 준용한다. <개정 2023.12.31>

2. 소득ㆍ세액 공제 대상금액의 표기에 관한 사항

1. 수입금액에서 다음 각 목의 금액의 합계액(수입금액을 초과하는 경우에는 그 초과하는 금액은 제외한다)을 공제한 금액을 그 소득금액(이하 이 조에서 "기준소득금액"이라 한다)으로 결정 또는 경정하는 방법. 다만, 기준소득금액이 제1호의2에 따른 소득금액에 재정경제부령으로 정하는 배율을 곱하여 계산한 금액 이상인 경우 2027년 12월 31일이 속하는 과세기간의 소득금액을 결정 또는 경정할 때까지는 그 배율을 곱하여 계산한 금액을 소득금액으로 결정할 수 있다.
[답변] 연간 소득금액 100만원 이하란 부양가족이 1년 동안 벌어들인 소득금액이 1백만원 이하임을 의미합니다.

[근거] 소득세법 제54조의2 제1항 및 제1항 각 호에 따른 부양가족 공제 요건에 관한 규정.

[결론 도출 과정] 부양가족 공제 요건 중 "연간 소득금액 100만원 이하"란, 해당 부양가족이 1년 동안 벌어들인 모든 소득(이자, 배당, 근로소득, 기타 소득 등)을 합산했을 때 그 금액이 1백만원 이하임을 의미하며, 이는 소득세법 제54조의2 제1항에서 규정하는 부양가족의 소득 기준에 해당합니다.

[출처]
ID:ids1


In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"

pipeline = MultiQueryPipeline(prompt_multi_query, llm, 25, 10, vectorstore)
print(Reference().final_output(query, pipeline))

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.\n각 쿼리는 반드시 줄바꿈으로 구분해.\n\n질문: 소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
  ]
}
[llm/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] [3.36s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. 소득세법상 부양가족 공제 대상의 연간 소득금액 산정 

In [ ]:
query = "간편장부를 기장하지 아니하였을 때 어떤 불이익이 있나요?"

pipeline = MultiQueryPipeline(prompt_multi_query, llm, 25, 10, vectorstore)
print(Reference().final_output(query, pipeline))

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "간편장부를 기장하지 아니하였을 때 어떤 불이익이 있나요?"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "간편장부를 기장하지 아니하였을 때 어떤 불이익이 있나요?"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.\n각 쿼리는 반드시 줄바꿈으로 구분해.\n\n질문: 간편장부를 기장하지 아니하였을 때 어떤 불이익이 있나요?"
  ]
}
[llm/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] [2.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. 간편장부를 미작성 또는 미기장할 경우, 법인세법상 어떤 불이익 또는 가산세 부과 사유가 발생하는지 구체적으로 설명해 주시기 바랍니다.\n\n2. 간편장부를 작성하지 아니하거나 

In [ ]:
query= "미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"

pipeline = MultiQueryPipeline(prompt_multi_query, llm, 25, 10, vectorstore)
print(Reference().final_output(query, pipeline))

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n질문을 세법 전문 용어를 사용해 3개의 서로 다른 query로 재작성해.\n각 쿼리는 반드시 줄바꿈으로 구분해.\n\n질문: 미혼인 본인이 보유한 1채를 임대하고, 부모님이 보유한 주택에서 거주하는 경우 임대소득세 과세대상인지?"
  ]
}
[llm/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] [1.75s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. 미혼자가 본인 명의로 임대한 부동산에